# Changing the Leetcode stats in my CV

## 1. get data from Leetcode

In [14]:
import requests
import json
from bs4 import BeautifulSoup

In [15]:

def get_leetcode_profile_data(username):
    url = f"https://leetcode-stats-api.herokuapp.com/{username}/"
    response = requests.get(url)

    # Check if the response status code is 200 (OK)
    if response.status_code == 200:
        # Decode the byte content to a string
        response_content = response.content.decode('utf-8')

        # Parse the JSON string into a dictionary
        profile_data = json.loads(response_content)

        return profile_data
    else:
        print("Failed to retrieve data.")
        return None


In [22]:
# TMP
total = 31
easy = 26
medium = 5
hard = 0
rank = 1907688

In [23]:
# username = 'josefpolasek'
# profile_data = get_leetcode_profile_data(username)
# print(profile_data)

# total = profile_data["totalSolved"]
# easy = profile_data["easySolved"]
# medium = profile_data["mediumSolved"]
# hard = profile_data["hardSolved"]
# rank = profile_data["ranking"]

print(f"Total Problems Solved: {total}")
print(f"Easy: {easy}")
print(f"Medium: {medium}")
print(f"Hard: {hard}")
print(f"Rank: {rank:,}")

Total Problems Solved: 31
Easy: 26
Medium: 5
Hard: 0
Rank: 1,907,688


## 2. Update the CV

In [105]:
import os
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Google Docs API setup
SCOPES = ['https://www.googleapis.com/auth/documents']
SERVICE_ACCOUNT_FILE = 'credentials.json'

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

service = build('docs', 'v1', credentials=credentials)

# The ID of the Google Docs document
DOCUMENT_ID = '1bCtSLi71qrT03579jhfNyS6SOvdSKh7RUtPJMBMAGtY'

# Fetch the document content
try:
    document = service.documents().get(documentId=DOCUMENT_ID).execute()
    print("Document content fetched successfully.")

    content = ""
    for element in document.get('body', {}).get('content', []):
        if 'paragraph' in element:
            for run in element['paragraph']['elements']:
                if 'textRun' in run:
                    content += run['textRun']['content']

    print("Document content:")
    print(content)
except Exception as e:
    print(f"An error occurred: {e}")


Document content fetched successfully.
Document content:
Josef Polasek
Python AI Developer | josefpolasek12@gmail.com | +420 607 277 739 | Leetcode: josefpolasek | GitHub: josefpolasek 
EDUCATION				
IU International University of Applied Sciences		             Berlin, Germany (remote) 
B.Sc. in Applied Artificial Intelligence		June 2024 – June 2026 (expected)
Coursework: …

Tampere University of Applied Sciences		                            Tampere, Finland 
B.Eng. in Software Engineering		                    Aug 2022 – June 2024
Transferred to IU after 2nd year
Coursework: Data Structures & Algorithms, DevOps, Basics of Web Development, Mobile App Development, Embedded Systems, Programming in C++, Programming in Java, Calculus, Linear Algebra, Mechanics, Finnish	

EXPERIENCE				
Granton AI				Prague, Czechia (remote)
Python AI Developer		                          Feb 2024 – present
Created Python scripts to construct a pipeline that retrieved performance data from the AWS cross-regio

In [106]:
import os
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Setup Google Docs API
SCOPES = ['https://www.googleapis.com/auth/documents']
SERVICE_ACCOUNT_FILE = 'credentials.json'

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

docs_service = build('docs', 'v1', credentials=credentials)

# The ID of the Google Docs template
TEMPLATE_DOCUMENT_ID = '1bCtSLi71qrT03579jhfNyS6SOvdSKh7RUtPJMBMAGtY'
# The ID of the document to be updated
DOCUMENT_ID = '1Wz3zdm6XzjnNgS4fwg6N-ATNc3Gc8aRkNfkxezKvF3o'

# Variables with fetched data
easy = 25
medium = 3
hard = 0

# Define the placeholders and their replacements
placeholders = {
    '{easy_solved}': str(easy),
    '{medium_solved}': str(medium),
    '{hard_solved}': str(hard)
}


# Step 1: Fetch the content of the template document
def get_template_content(template_id):
    template = docs_service.documents().get(documentId=template_id).execute()
    content = template.get('body').get('content')
    return content


# Step 2: Replace placeholders in the template content
def replace_placeholders(content, placeholders):
    replaced_content = []
    for element in content:
        if 'paragraph' in element:
            for para_element in element['paragraph']['elements']:
                if 'textRun' in para_element:
                    text = para_element['textRun']['content']
                    for placeholder, replacement in placeholders.items():
                        text = text.replace(placeholder, replacement)
                    para_element['textRun']['content'] = text
        replaced_content.append(element)
    return replaced_content


# Step 3: Clear the existing document content
def clear_document_content(document_id):
    # Fetch the document to get its length
    document = docs_service.documents().get(documentId=document_id).execute()
    body_content = document.get('body').get('content')

    if len(body_content) > 1:  # Check if there's content to delete
        end_index = body_content[-1]['endIndex']

        requests = [
            {
                'deleteContentRange': {
                    'range': {
                        'startIndex': 1,
                        'endIndex': end_index - 1
                    }
                }
            }
        ]
        docs_service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
    else:
        print("Document is already empty, no content to delete.")


# Step 4: Insert new content into the existing document
def insert_new_content(document_id, content):
    requests = [
        {
            'insertText': {
                'location': {
                    'index': 1
                },
                'text': ''
            }
        }
    ]
    docs_service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()

    for element in content:
        requests.append({
            'insertText': {
                'location': {
                    'index': 1
                },
                'text': element['paragraph']['elements'][0]['textRun']['content']
            }
        })
    docs_service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()

In [107]:
try:
    # Fetch and replace content
    template_content = get_template_content(TEMPLATE_DOCUMENT_ID)
    replaced_content = replace_placeholders(template_content, placeholders)

    # Clear existing document content
    clear_document_content(DOCUMENT_ID)

    # Insert new content
    insert_new_content(DOCUMENT_ID, replaced_content)

    print("Document updated successfully.")
except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: <HttpError 400 when requesting https://docs.googleapis.com/v1/documents/1Wz3zdm6XzjnNgS4fwg6N-ATNc3Gc8aRkNfkxezKvF3o:batchUpdate?alt=json returned "Invalid requests[0].insertText: Insert text requests must specify text to insert.". Details: "Invalid requests[0].insertText: Insert text requests must specify text to insert.">


In [131]:
import os
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Setup Google Docs API
SCOPES = ['https://www.googleapis.com/auth/documents']
SERVICE_ACCOUNT_FILE = 'credentials.json'

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

docs_service = build('docs', 'v1', credentials=credentials)

# The ID of the Google Docs template
TEMPLATE_DOCUMENT_ID = '1bCtSLi71qrT03579jhfNyS6SOvdSKh7RUtPJMBMAGtY'
# The ID of the document to be updated
DOCUMENT_ID = '1Wz3zdm6XzjnNgS4fwg6N-ATNc3Gc8aRkNfkxezKvF3o'

# Variables with fetched data
easy = 25
medium = 3
hard = 0

# Define the placeholders and their replacements
placeholders = {
    '{easy_solved}': str(easy),
    '{medium_solved}': str(medium),
    '{hard_solved}': str(hard)
}


# Step 1: Fetch the content of the template document
def get_template_content(template_id):
    template = docs_service.documents().get(documentId=template_id).execute()
    content = template.get('body').get('content')
    return content


# Step 2: Replace placeholders in the template content
def replace_placeholders(content, placeholders):
    replaced_content = []
    for element in content:
        if 'paragraph' in element:
            for para_element in element['paragraph']['elements']:
                if 'textRun' in para_element:
                    text = para_element['textRun']['content']
                    for placeholder, replacement in placeholders.items():
                        text = text.replace(placeholder, replacement)
                    para_element['textRun']['content'] = text
        replaced_content.append(element)
    return replaced_content


# Step 3: Clear the existing document content
def clear_document_content(document_id):
    # Fetch the document to get its length
    document = docs_service.documents().get(documentId=document_id).execute()
    body_content = document.get('body').get('content')

    if len(body_content) > 1:  # Check if there's content to delete
        end_index = body_content[-1]['endIndex']

        requests = [
            {
                'deleteContentRange': {
                    'range': {
                        'startIndex': 1,
                        'endIndex': end_index - 1
                    }
                }
            }
        ]
        docs_service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
    else:
        print("Document is already empty, no content to delete.")


# Step 4: Insert new content into the existing document
def insert_new_content(document_id, content):
    requests = []

    for element in content:
        if 'paragraph' in element:
            for para_element in element['paragraph']['elements']:
                if 'textRun' in para_element:
                    text = para_element['textRun']['content']
                    if text.strip():  # Only add non-empty text
                        requests.append({
                            'insertText': {
                                'location': {
                                    'index': 1
                                },
                                'text': text
                            }
                        })

    if requests:
        docs_service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
    else:
        print("No content to insert.")

In [126]:
template_content = get_template_content(TEMPLATE_DOCUMENT_ID)
template_content

[{'endIndex': 1,
  'sectionBreak': {'sectionStyle': {'columnSeparatorStyle': 'NONE',
    'contentDirection': 'LEFT_TO_RIGHT',
    'sectionType': 'CONTINUOUS'}}},
 {'startIndex': 1,
  'endIndex': 15,
  'paragraph': {'elements': [{'startIndex': 1,
     'endIndex': 6,
     'textRun': {'content': 'Josef',
      'textStyle': {'bold': True,
       'fontSize': {'magnitude': 16, 'unit': 'PT'},
       'weightedFontFamily': {'fontFamily': 'Calibri', 'weight': 400}}}},
    {'startIndex': 6,
     'endIndex': 7,
     'textRun': {'content': ' ',
      'textStyle': {'bold': True,
       'italic': False,
       'underline': False,
       'strikethrough': False,
       'smallCaps': False,
       'backgroundColor': {},
       'foregroundColor': {'color': {'rgbColor': {}}},
       'fontSize': {'magnitude': 16, 'unit': 'PT'},
       'weightedFontFamily': {'fontFamily': 'Calibri', 'weight': 400},
       'baselineOffset': 'NONE'}}},
    {'startIndex': 7,
     'endIndex': 14,
     'textRun': {'content': 'Pol

In [111]:
replaced_content = replace_placeholders(template_content, placeholders)
replaced_content

[{'endIndex': 1,
  'sectionBreak': {'sectionStyle': {'columnSeparatorStyle': 'NONE',
    'contentDirection': 'LEFT_TO_RIGHT',
    'sectionType': 'CONTINUOUS'}}},
 {'startIndex': 1,
  'endIndex': 15,
  'paragraph': {'elements': [{'startIndex': 1,
     'endIndex': 6,
     'textRun': {'content': 'Josef',
      'textStyle': {'bold': True,
       'fontSize': {'magnitude': 16, 'unit': 'PT'},
       'weightedFontFamily': {'fontFamily': 'Calibri', 'weight': 400}}}},
    {'startIndex': 6,
     'endIndex': 7,
     'textRun': {'content': ' ',
      'textStyle': {'bold': True,
       'italic': False,
       'underline': False,
       'strikethrough': False,
       'smallCaps': False,
       'backgroundColor': {},
       'foregroundColor': {'color': {'rgbColor': {}}},
       'fontSize': {'magnitude': 16, 'unit': 'PT'},
       'weightedFontFamily': {'fontFamily': 'Calibri', 'weight': 400},
       'baselineOffset': 'NONE'}}},
    {'startIndex': 7,
     'endIndex': 14,
     'textRun': {'content': 'Pol

In [135]:
clear_document_content(DOCUMENT_ID)

In [136]:
insert_new_content(DOCUMENT_ID, template_content)

1. Copy the document content

In [145]:
def get_document_content(document_id):
    document = service.documents().get(documentId=document_id).execute()
    return document

source_document_id = '1bCtSLi71qrT03579jhfNyS6SOvdSKh7RUtPJMBMAGtY'
source_document = get_document_content(source_document_id)
# source_document

content = source_document.get('body').get('content')
# content


2. Insert content

In [155]:
def insert_content(destination_document_id, requests):
    result = service.documents().batchUpdate(
        documentId=destination_document_id,
        body={'requests': requests}
    ).execute()
    return result

destination_document_id = '1Wz3zdm6XzjnNgS4fwg6N-ATNc3Gc8aRkNfkxezKvF3o'

requests = [
    {'insertText': {'location': {'index': 1}, 'text': ''}}
]

for element in content:
    requests.append({'insertText': {'location': {'index': 1}, 'text': element.get('textRun', {}).get('content', '')}})
    
insert_content(destination_document_id, requests)

HttpError: <HttpError 400 when requesting https://docs.googleapis.com/v1/documents/1Wz3zdm6XzjnNgS4fwg6N-ATNc3Gc8aRkNfkxezKvF3o:batchUpdate?alt=json returned "Invalid requests[0].insertText: Insert text requests must specify text to insert.". Details: "Invalid requests[0].insertText: Insert text requests must specify text to insert.">

In [150]:

for element in content:
    requests.append({'insertText': {'location': {'index': 1}, 'text': element.get('textRun', {}).get('content', '')}})

insert_content(destination_document_id, requests)


HttpError: <HttpError 400 when requesting https://docs.googleapis.com/v1/documents/1Wz3zdm6XzjnNgS4fwg6N-ATNc3Gc8aRkNfkxezKvF3o:batchUpdate?alt=json returned "Invalid requests[0].insertText: Insert text requests must specify text to insert.". Details: "Invalid requests[0].insertText: Insert text requests must specify text to insert.">

In [147]:

for element in content:
    requests.append({'insertText': {'location': {'index': 1}, 'text': element.get('textRun', {}).get('content', '')}})

insert_content(destination_document_id, requests)


HttpError: <HttpError 400 when requesting https://docs.googleapis.com/v1/documents/1Wz3zdm6XzjnNgS4fwg6N-ATNc3Gc8aRkNfkxezKvF3o:batchUpdate?alt=json returned "Invalid requests[0].insertText: Insert text requests must specify text to insert.". Details: "Invalid requests[0].insertText: Insert text requests must specify text to insert.">